学习R语言{lmtest}包，该包提供了许多实用的检验函数，并且可以与{sandwich}包配合使用，威力无穷。

In [20]:
library(lmtest)
library(broom)
library(tidyverse)

Loading required package: zoo




Attaching package: 'zoo'


The following objects are masked from 'package:base':

    as.Date, as.Date.numeric


-- Attaching core tidyverse packages ------------------------ tidyverse 2.0.0 --
v dplyr     1.1.4     v readr     2.1.5
v forcats   1.0.0     v stringr   1.5.1
v ggplot2   3.5.1     v tibble    3.2.1
v lubridate 1.9.4     v tidyr     1.3.1
v purrr     1.0.2     
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
i Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


# 通用检验

## coeftest

用多了，不想介绍。

## lrtest

In [26]:
## 使用 Greene (1993) 的数据进行示例：

## 加载数据并计算滞后项
data("USDistLag")
usdl <- na.contiguous(cbind(USDistLag, stats::lag(USDistLag, k = -1)))
colnames(usdl) <- c("con", "gnp", "con1", "gnp1")

## 构建两个嵌套模型
fm1 <- lm(con ~ gnp + gnp1, data = usdl)
fm2 <- lm(con ~ gnp + con1 + gnp1, data = usdl)

## 多种等价形式的似然比检验
lrtest(fm2, fm1)                    # 显式比较两个模型
lrtest(fm2, 2)                      # 指定删除第2个解释变量（con1）
lrtest(fm2, "con1")                 # 指定按变量名删除 con1
lrtest(fm2, . ~ . - con1)           # 使用公式语法删除 con1


,#Df,LogLik,Df,Chisq,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,5,-56.06902,NA,NA,NA
2,4,-65.87133,-1,19.60463,9.523829e-06


,#Df,LogLik,Df,Chisq,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,5,-56.06902,NA,NA,NA
2,4,-65.87133,-1,19.60463,9.523829e-06


,#Df,LogLik,Df,Chisq,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,5,-56.06902,NA,NA,NA
2,4,-65.87133,-1,19.60463,9.523829e-06


,#Df,LogLik,Df,Chisq,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,5,-56.06902,NA,NA,NA
2,4,-65.87133,-1,19.60463,9.523829e-06


# 异方差检验

## bptest

In [14]:
## generate a regressor
x <- rep(c(-1,1), 50) 

## generate heteroskedastic and homoskedastic disturbances
err1 <- rnorm(100, sd = rep(c(1, 2), 50))
err2 <- rnorm(100)

## generate a linear relationship
y1 <- 1 + x + err1
y2 <- 1 + x + err2

## perform Breusch-Pagan test
bptest(y1 ~ x)
bptest(y2 ~ x)


	studentized Breusch-Pagan test

data:  y1 ~ x
BP = 15.061, df = 1, p-value = 0.0001041



	studentized Breusch-Pagan test

data:  y2 ~ x
BP = 0.2176, df = 1, p-value = 0.6409


## waldtest

In [28]:
# 测试Wald检验
full  <- lm(data = mtcars, mpg ~ wt + hp + qsec)
reduced <- lm(data = mtcars, mpg ~ wt + hp)
waldtest(full, reduced)

,Res.Df,Df,F,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>
1,28,NA,NA,NA
2,29,-1,1.35267,0.2546284


## GQ检验

In [19]:
## generate a regressor
x <- rep(c(-1, 1), 50)

## generate heteroskedastic and homoskedastic disturbances
err1 <- c(rnorm(50, sd = 1), rnorm(50, sd = 2))
err2 <- rnorm(100)

## generate a linear relationship
y1 <- 1 + x + err1
y2 <- 1 + x + err2

## perform Goldfeld-Quandt test
gqtest(y1 ~ x)
gqtest(y2 ~ x)


	Goldfeld-Quandt test

data:  y1 ~ x
GQ = 4.2247, df1 = 48, df2 = 48, p-value = 9.036e-07
alternative hypothesis: variance increases from segment 1 to 2



	Goldfeld-Quandt test

data:  y2 ~ x
GQ = 0.94711, df1 = 48, df2 = 48, p-value = 0.5743
alternative hypothesis: variance increases from segment 1 to 2


## hmctest

In [24]:
## 生成解释变量
x <- rep(c(-1, 1), 50)

## 生成异方差和同方差扰动项
err1 <- c(rnorm(50, sd = 1), rnorm(50, sd = 2))  # 异方差
err2 <- rnorm(100)                              # 同方差

## 构造线性关系
y1 <- 1 + x + err1
y2 <- 1 + x + err2

## 执行 Harrison-McCabe 异方差检验
hmctest(y1 ~ x)
hmctest(y2 ~ x)



	Harrison-McCabe test

data:  y1 ~ x
HMC = 0.18949, p-value < 2.2e-16



	Harrison-McCabe test

data:  y2 ~ x
HMC = 0.59699, p-value = 0.898


# 自相关检验

## bgtest

In [ ]:
## Generate a stationary and an AR(1) series
x <- rep(c(1, -1), 50)
y1 <- 1 + x + rnorm(100)

In [9]:
## Perform Breusch-Godfrey test for first-order serial correlation:
bgtest(y1 ~ x)


	Breusch-Godfrey test for serial correlation of order up to 1

data:  y1 ~ x
LM test = 0.14108, df = 1, p-value = 0.7072


In [10]:
## or for fourth-order serial correlation
bgtest(y1 ~ x, order = 4)


	Breusch-Godfrey test for serial correlation of order up to 4

data:  y1 ~ x
LM test = 0.61713, df = 4, p-value = 0.9611


In [ ]:
y2 <- stats::filter(y1, 0.5, method = "recursive")
bgtest(y2 ~ x)

bg4 <- bgtest(y2 ~ x, order = 4)
coeftest(bg4)


	Breusch-Godfrey test for serial correlation of order up to 1

data:  y2 ~ x
LM test = 22.97, df = 1, p-value = 1.646e-06



z test of coefficients:

              Estimate Std. Error z value  Pr(>|z|)    
(Intercept)  -0.012657   0.103809 -0.1219    0.9030    
x            -0.009068   0.103631 -0.0875    0.9303    
lag(resid)_1  0.458501   0.103272  4.4398 9.006e-06 ***
lag(resid)_2  0.039944   0.116206  0.3437    0.7310    
lag(resid)_3  0.058075   0.116052  0.5004    0.6168    
lag(resid)_4 -0.024049   0.108217 -0.2222    0.8241    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


## dwtest

In [15]:
## generate two AR(1) error terms with parameter
## rho = 0 (white noise) and rho = 0.9 respectively
err1 <- rnorm(100)

## generate regressor and dependent variable
x <- rep(c(-1,1), 50)
y1 <- 1 + x + err1

## perform Durbin-Watson test
dwtest(y1 ~ x)


	Durbin-Watson test

data:  y1 ~ x
DW = 1.8453, p-value = 0.2481
alternative hypothesis: true autocorrelation is greater than 0


In [17]:
err2 <- stats::filter(err1, 0.9, method="recursive")

y2 <- 1 + x + err2
dwtest(y2 ~ x)


	Durbin-Watson test

data:  y2 ~ x
DW = 0.31312, p-value < 2.2e-16
alternative hypothesis: true autocorrelation is greater than 0


# grangertest

In [20]:
## Which came first: the chicken or the egg?
data(ChickEgg)
grangertest(egg ~ chicken, order = 3, data = ChickEgg)
grangertest(chicken ~ egg, order = 3, data = ChickEgg)

,Res.Df,Df,F,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>
1,44,NA,NA,NA
2,47,-3,0.5916153,0.6237862


,Res.Df,Df,F,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>
1,44,NA,NA,NA
2,47,-3,5.404984,0.002966397


In [21]:
## alternative ways of specifying the same test
grangertest(ChickEgg, order = 3)
grangertest(ChickEgg[, 1], ChickEgg[, 2], order = 3)

,Res.Df,Df,F,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>
1,44,NA,NA,NA
2,47,-3,0.5916153,0.6237862


,Res.Df,Df,F,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>
1,44,NA,NA,NA
2,47,-3,0.5916153,0.6237862


# 线性设定检验

## harvtest

In [22]:
# generate a regressor and dependent variable
x <- 1:50
y1 <- 1 + x + rnorm(50)
y2 <- y1 + 0.3*x^2

## perform Harvey-Collier test
harv <- harvtest(y1 ~ x)
harv


	Harvey-Collier test

data:  y1 ~ x
HC = 0.51479, df = 47, p-value = 0.6091


In [23]:
## calculate critical value vor 0.05 level
qt(0.95, harv$parameter)
harvtest(y2 ~ x)

[1] 1.677927


	Harvey-Collier test

data:  y2 ~ x
HC = 7.9559, df = 47, p-value = 2.964e-10


## petest

In [28]:
if (require("AER")) {

  ## Verbeek (2004), Section 3 示例数据
  data("HousePrices", package = "AER")

  ## 表 3.3：线性模型
  hp_lin <- lm(price ~ ., data = HousePrices)
  summary(hp_lin)

  ## 表 3.2：对数线性模型（对 lotsize 取对数）
  hp_log <- update(hp_lin, log(price) ~ . - lotsize + log(lotsize))
  summary(hp_log)

  ## PE 检验：线性 vs 对数线性
  petest(hp_lin, hp_log)

  ## Greene (2003), Example 9.8 示例数据
  data("USMacroG", package = "AER")

  ## 表 9.2：线性与对数线性模型
  usm_lin <- lm(m1 ~ tbill + gdp, data = USMacroG)
  usm_log <- lm(log(m1) ~ log(tbill) + log(gdp), data = USMacroG)

  ## PE 检验：对比两个规格，结果与 Greene 勘误表一致
  petest(usm_lin, usm_log)
}


,Estimate,Std. Error,t value,Pr(>|t|)
,<dbl>,<dbl>,<dbl>,<dbl>
M1 + log(fit(M1))-fit(M2),-2.093544e+02,26.758025405,-7.8239849,2.900156e-13
M2 + fit(M1)-exp(fit(M2)),-4.188803e-05,0.000261327,-0.1602897,8.728146e-01


## raintest

In [29]:
## 构造非线性数据
x <- 1:30
y <- x^2 + rnorm(30, 0, 2)

## 执行 Rainbow 检验
rain <- raintest(y ~ x)
rain

## 查找临界值（用于手动比较检验统计量）
qf(0.95, rain$parameter[1], rain$parameter[2])


	Rainbow test

data:  y ~ x
Rain = 28.008, df1 = 15, df2 = 13, p-value = 1.899e-07


[1] 2.53311

## resettest

In [21]:
x <- c(1:30)
y1 <- 1 + x + x^2 + rnorm(30)
y2 <- 1 + x + rnorm(30)

resettest(y1 ~ x, power=2, type="regressor")
resettest(y2 ~ x, power=2, type="regressor")


	RESET test

data:  y1 ~ x
RESET = 130207, df1 = 1, df2 = 27, p-value < 2.2e-16



	RESET test

data:  y2 ~ x
RESET = 0.027194, df1 = 1, df2 = 27, p-value = 0.8702
